In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

26931


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
26926,2021-08-06,47,沖縄県,565,27619,1,239
26927,2021-08-07,47,沖縄県,548,28167,0,239
26928,2021-08-08,47,沖縄県,574,28741,0,239
26929,2021-08-09,47,沖縄県,332,29073,1,240
26930,2021-08-10,47,沖縄県,332,29405,2,242


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,12606,12600,157
43,香川県,21201,2637,32
44,高知県,22347,2194,29
45,鳥取県,17763,1011,3
46,鹿児島県,26358,4569,41


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
26924,2021-08-04,47,沖縄県,602,26406,0,237,2858.0
26925,2021-08-05,47,沖縄県,648,27054,1,238,3114.0
26926,2021-08-06,47,沖縄県,565,27619,1,239,3297.0
26927,2021-08-07,47,沖縄県,548,28167,0,239,3406.0
26928,2021-08-08,47,沖縄県,574,28741,0,239,3613.0
26929,2021-08-09,47,沖縄県,332,29073,1,240,3736.0
26930,2021-08-10,47,沖縄県,332,29405,2,242,3601.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
26921,2021-08-01,47,沖縄県,367,25128,0,236,2397.0,-0.164009,0.239479,0.755981
26922,2021-08-02,47,沖縄県,209,25337,0,236,2490.0,-0.430518,0.241544,0.801724
26923,2021-08-03,47,沖縄県,467,25804,1,237,2603.0,1.234450,0.124791,0.319209
26924,2021-08-04,47,沖縄県,602,26406,0,237,2858.0,0.289079,0.168913,0.734870
26925,2021-08-05,47,沖縄県,648,27054,1,238,3114.0,0.076412,0.161303,0.653061
26926,2021-08-06,47,沖縄県,565,27619,1,239,3297.0,-0.128086,0.146649,0.479058
26927,2021-08-07,47,沖縄県,548,28167,0,239,3406.0,-0.030088,0.121034,0.248292
26928,2021-08-08,47,沖縄県,574,28741,0,239,3613.0,0.047445,0.151242,0.564033
26929,2021-08-09,47,沖縄県,332,29073,1,240,3736.0,-0.421603,0.152516,0.588517
26930,2021-08-10,47,沖縄県,332,29405,2,242,3601.0,0.000000,-0.023834,-0.289079


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
572,2021-08-10,北海道,347,1,2303.0,0.119355,0.105159,0.644550
1145,2021-08-10,青森県,26,0,158.0,1.363636,0.140990,0.000000
1718,2021-08-10,岩手県,26,0,159.0,1.000000,0.184741,0.238095
2291,2021-08-10,宮城県,87,0,657.0,0.474576,0.061139,0.175676
2864,2021-08-10,秋田県,8,0,74.0,-0.111111,0.126788,0.333333


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,13752,6399,114,2021-08-10,55,0,501.0,-0.112903,0.006872,-0.179104
1,京都府,14898,20629,249,2021-08-10,107,0,1822.0,-0.605166,-0.017578,-0.436842
2,佐賀県,23493,3058,25,2021-08-10,52,0,257.0,1.166667,0.244085,1.736842
3,兵庫県,16044,47716,1321,2021-08-10,228,1,2807.0,-0.170909,-0.075383,-0.482993
4,北海道,573,47422,1429,2021-08-10,347,1,2303.0,0.119355,0.105159,0.644550


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,13752,6399,114,2021-08-10,55,0,501.0,-0.112903,0.006872,-0.179104,1813859
1,京都府,14898,20629,249,2021-08-10,107,0,1822.0,-0.605166,-0.017578,-0.436842,2545899
2,佐賀県,23493,3058,25,2021-08-10,52,0,257.0,1.166667,0.244085,1.736842,823810
3,兵庫県,16044,47716,1321,2021-08-10,228,1,2807.0,-0.170909,-0.075383,-0.482993,5549568
4,北海道,573,47422,1429,2021-08-10,347,1,2303.0,0.119355,0.105159,0.644550,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,573,47422,1429,2021-08-10,347,1,2303.0,0.119355,0.105159,0.644550,5267762,90.023050,900.230496,43.718756
41,青森県,1146,2909,32,2021-08-10,26,0,158.0,1.363636,0.140990,0.000000,1275783,22.801683,228.016834,12.384551
19,岩手県,1719,2205,47,2021-08-10,26,0,159.0,1.000000,0.184741,0.238095,1235517,17.846780,178.467799,12.869107
11,宮城県,2292,10705,91,2021-08-10,87,0,657.0,0.474576,0.061139,0.175676,2292385,46.698090,466.980895,28.660107
36,秋田県,2865,1104,21,2021-08-10,8,0,74.0,-0.111111,0.126788,0.333333,985416,11.203390,112.033902,7.509519
15,山形県,3438,2393,47,2021-08-10,15,0,193.0,-0.210526,-0.058628,-0.531250,1082296,22.110402,221.104023,17.832460
35,福島県,4011,6619,162,2021-08-10,52,0,595.0,-0.395349,0.043987,-0.054545,1881981,35.170387,351.703869,31.615622
38,茨城県,4584,14784,172,2021-08-10,223,0,1753.0,0.027650,0.066530,0.131980,2921436,50.605250,506.052503,60.004737
27,栃木県,5157,9510,90,2021-08-10,101,1,909.0,0.063158,-0.017180,-0.273381,1965516,48.384241,483.842411,46.247398
37,群馬県,5730,9993,155,2021-08-10,102,0,911.0,0.120879,-0.009670,-0.301370,1969439,50.740338,507.403377,46.256827


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      22363.829787
std       43670.245689
min         827.000000
25%        2997.500000
50%        6399.000000
75%       13874.000000
max      254781.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,7449,254781,2317,2021-08-10,2612,3,27851.0,-0.094313,-0.036323,-0.295767,13834925,184.157847,1841.578469,201.309367
1,大阪府,15471,124150,2733,2021-08-10,697,0,7598.0,-0.299497,-0.046104,-0.354032,8849635,140.288272,1402.882718,85.856648
2,神奈川県,8022,100491,1014,2021-08-10,1572,5,12902.0,-0.274238,0.042440,0.211094,9209442,109.117360,1091.173602,140.095350
3,埼玉県,6303,68446,857,2021-08-10,1166,2,8793.0,0.005172,0.020178,0.107312,7390054,92.619080,926.190796,118.984246
4,千葉県,6876,58209,737,2021-08-10,859,0,6713.0,-0.097689,0.017289,0.099872,6319772,92.106171,921.061709,106.222186
5,愛知県,13179,57206,1013,2021-08-10,259,0,2476.0,0.011719,0.026580,0.003876,7575530,75.514188,755.141884,32.684182
6,兵庫県,16044,47716,1321,2021-08-10,228,1,2807.0,-0.170909,-0.075383,-0.482993,5549568,85.981467,859.814674,50.580514
7,北海道,573,47422,1429,2021-08-10,347,1,2303.0,0.119355,0.105159,0.644550,5267762,90.023050,900.230496,43.718756
8,福岡県,22920,45505,540,2021-08-10,458,2,4833.0,-0.266026,0.007249,-0.101961,5129841,88.706453,887.064531,94.213446
9,沖縄県,26931,29405,242,2021-08-10,332,2,3601.0,0.000000,-0.023834,-0.289079,1481547,198.474972,1984.749724,243.056751


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      52.394860
std       40.727190
min       11.203390
25%       26.196755
50%       35.531994
75%       61.344368
max      198.474972
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,26931,29405,242,2021-08-10,332,2,3601.0,0.000000,-0.023834,-0.289079,1481547,198.474972,1984.749724,243.056751
26,東京都,7449,254781,2317,2021-08-10,2612,3,27851.0,-0.094313,-0.036323,-0.295767,13834925,184.157847,1841.578469,201.309367
32,神奈川県,8022,100491,1014,2021-08-10,1572,5,12902.0,-0.274238,0.042440,0.211094,9209442,109.117360,1091.173602,140.095350
7,埼玉県,6303,68446,857,2021-08-10,1166,2,8793.0,0.005172,0.020178,0.107312,7390054,92.619080,926.190796,118.984246
5,千葉県,6876,58209,737,2021-08-10,859,0,6713.0,-0.097689,0.017289,0.099872,6319772,92.106171,921.061709,106.222186
34,福岡県,22920,45505,540,2021-08-10,458,2,4833.0,-0.266026,0.007249,-0.101961,5129841,88.706453,887.064531,94.213446
9,大阪府,15471,124150,2733,2021-08-10,697,0,7598.0,-0.299497,-0.046104,-0.354032,8849635,140.288272,1402.882718,85.856648
1,京都府,14898,20629,249,2021-08-10,107,0,1822.0,-0.605166,-0.017578,-0.436842,2545899,81.028352,810.283519,71.566075
38,茨城県,4584,14784,172,2021-08-10,223,0,1753.0,0.027650,0.066530,0.131980,2921436,50.605250,506.052503,60.004737
3,兵庫県,16044,47716,1321,2021-08-10,228,1,2807.0,-0.170909,-0.075383,-0.482993,5549568,85.981467,859.814674,50.580514


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    46.000000
mean      0.042695
std       0.088340
min      -0.104331
25%      -0.017478
50%       0.023466
75%       0.090789
max       0.329932
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
20,島根県,18336,827,2,2021-08-10,17,0,68.0,1.125000,0.329932,0.700000,679324,12.173867,121.738670,10.009951
2,佐賀県,23493,3058,25,2021-08-10,52,0,257.0,1.166667,0.244085,1.736842,823810,37.120210,371.202098,31.196514
14,山口県,20055,3496,80,2021-08-10,32,0,172.0,0.600000,0.192663,1.133333,1369882,25.520446,255.204463,12.555826
19,岩手県,1719,2205,47,2021-08-10,26,0,159.0,1.000000,0.184741,0.238095,1235517,17.846780,178.467799,12.869107
13,富山県,9168,2649,38,2021-08-10,35,0,309.0,0.060606,0.150289,0.842105,1055999,25.085251,250.852510,29.261391
46,鹿児島県,26358,4569,41,2021-08-10,82,1,505.0,0.242424,0.142953,0.822222,1630146,28.028164,280.281643,30.978820
41,青森県,1146,2909,32,2021-08-10,26,0,158.0,1.363636,0.140990,0.000000,1275783,22.801683,228.016834,12.384551
36,秋田県,2865,1104,21,2021-08-10,8,0,74.0,-0.111111,0.126788,0.333333,985416,11.203390,112.033902,7.509519
22,徳島県,20628,1871,63,2021-08-10,8,0,60.0,0.333333,0.125572,0.142857,742505,25.198484,251.984835,8.080754
8,大分県,25212,4009,64,2021-08-10,39,0,283.0,0.026316,0.121376,0.772727,1151229,34.823654,348.236537,24.582425


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
45,鳥取県,17763,1011,3,2021-08-10,6,0,173.0,-0.647059,-0.104331,-0.760000,561175,18.015770,180.157705,30.828173
3,兵庫県,16044,47716,1321,2021-08-10,228,1,2807.0,-0.170909,-0.075383,-0.482993,5549568,85.981467,859.814674,50.580514
31,石川県,9741,5836,119,2021-08-10,33,0,510.0,-0.535211,-0.063845,-0.611765,1139612,51.210412,512.104120,44.752074
15,山形県,3438,2393,47,2021-08-10,15,0,193.0,-0.210526,-0.058628,-0.531250,1082296,22.110402,221.104023,17.832460
33,福井県,10314,1902,36,2021-08-10,21,0,194.0,-0.300000,-0.055940,-0.511628,780053,24.382959,243.829586,24.870105
9,大阪府,15471,124150,2733,2021-08-10,697,0,7598.0,-0.299497,-0.046104,-0.354032,8849635,140.288272,1402.882718,85.856648
6,和歌山県,17190,3272,50,2021-08-10,19,0,195.0,-0.050000,-0.045450,-0.424242,954258,34.288421,342.884210,20.434725
26,東京都,7449,254781,2317,2021-08-10,2612,3,27851.0,-0.094313,-0.036323,-0.295767,13834925,184.157847,1841.578469,201.309367
12,宮崎県,25785,3529,27,2021-08-10,23,0,236.0,-0.281250,-0.030444,-0.303030,1095903,32.201755,322.017551,21.534753
28,沖縄県,26931,29405,242,2021-08-10,332,2,3601.0,0.000000,-0.023834,-0.289079,1481547,198.474972,1984.749724,243.056751


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

#plt.style.use("ggplot")
### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
for i,v in enumerate(chart_data["各地の感染者数_1日ごとの発表数"]):
    plt.text(v,i,"{:,}".format(v),va="center",fontsize=7)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
for i,v in enumerate(chart_data["直近1週間の10万人当たりの感染者数"]):
    plt.text(v,i,"{:,}".format(int(v)),va="center",fontsize=7)
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  5  1  2  0  0  0  1]
[   827.    32571.25  64315.5   96059.75 127804.   159548.25 191292.5
 223036.75 254781.  ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([36.,  2.,  6.,  0.,  1.,  1.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([ 4., 21., 10.,  2.,  6.,  1.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [27]:
pref[pref["都道府県名"]=="山口県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
20045,2021-08-01,35,山口県,9,3306,0,80,95.0,-0.470588,0.177624,0.800000
20046,2021-08-02,35,山口県,3,3309,0,80,91.0,-0.666667,0.025243,-0.571429
20047,2021-08-03,35,山口県,15,3324,0,80,97.0,4.000000,0.555855,0.666667
20048,2021-08-04,35,山口県,27,3351,0,80,114.0,0.800000,0.654268,1.700000
20049,2021-08-05,35,山口県,17,3368,0,80,115.0,-0.370370,0.515643,0.062500
20050,2021-08-06,35,山口県,21,3389,0,80,109.0,0.235294,0.451043,-0.222222
20051,2021-08-07,35,山口県,31,3420,0,80,123.0,0.476190,0.571980,0.823529
20052,2021-08-08,35,山口県,24,3444,0,80,138.0,-0.225806,0.606949,1.666667
20053,2021-08-09,35,山口県,20,3464,0,80,155.0,-0.166667,0.678377,5.666667
20054,2021-08-10,35,山口県,32,3496,0,80,172.0,0.600000,0.192663,1.133333
